<a href="https://colab.research.google.com/github/navnee7h/AI-generated-Text-Detection/blob/LSVM-Model/AI_detector_LSVM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🔘AI Generated text detection ML model**

## **🔗Linking  google drive to google colab**

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
!pwd

/content


# **Load data**

In [51]:
import pandas as pd
df = pd.read_csv("drive/My Drive/GPT-wiki-intro.csv")
df

,id,url,title,wiki_intro,generated_intro,title_len,wiki_intro_len,generated_intro_len,prompt,generated_text,prompt_tokens,generated_text_tokens
0,63064638,https://en.wikipedia.org/wiki/Sexhow%20railway...,Sexhow railway station,Sexhow railway station was a railway station b...,Sexhow railway station was a railway station l...,3,174,78,200 word wikipedia style introduction on 'Sexh...,"located in the town of Sexhow, on the Cumbria...",25,88
1,279621,https://en.wikipedia.org/wiki/Eti%C3%A4inen,Etiäinen,"In Finnish folklore, all places and things, an...","In Finnish folklore, all places and things, an...",1,187,80,200 word wikipedia style introduction on 'Etiä...,"animate or inanimate, have a spirit or ""etiäi...",26,101
2,287229,https://en.wikipedia.org/wiki/Inverse%20functi...,Inverse function theorem,"In mathematics, specifically differential calc...","In mathematics, specifically differential calc...",3,170,59,200 word wikipedia style introduction on 'Inve...,function theorem states that for every real-v...,26,65
3,26712375,https://en.wikipedia.org/wiki/Stepping%20on%20...,Stepping on Roses,is a Japanese shōjo manga series written and i...,is a Japanese shōjo manga series written and i...,3,335,121,200 word wikipedia style introduction on 'Step...,and illustrated by Maki Fujii. The series fol...,26,150
4,38894426,https://en.wikipedia.org/wiki/Rob%20Bradley,Rob Bradley,"Robert Milner ""Rob"" Bradley, Jr. (born August ...","Robert Milner ""Rob"" Bradley, Jr. (born August ...",2,170,136,200 word wikipedia style introduction on 'Rob ...,"29, 1973) is an American former professional ...",28,162
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,44173767,https://en.wikipedia.org/wiki/Randy%20Borum,Randy Borum,Randy Borum is a Professor and Coordinator of ...,Randy Borum is a Professor and Coordinator of ...,2,185,71,200 word wikipedia style introduction on 'Rand...,of the Master of Fine Arts Program in Creativ...,25,92
149996,33564134,https://en.wikipedia.org/wiki/Sa%27och%20language,Sa'och language,"Sa'och (, also, ""Sauch"") is an endangered, nea...","Sa'och (, also, ""Sauch"") is an endangered, nuc...",2,175,134,200 word wikipedia style introduction on 'Sa'o...,"nuclear-speaking, isolate language of the Ath...",33,184
149997,4219548,https://en.wikipedia.org/wiki/Philip%20Hanawalt,Philip Hanawalt,Philip C. Hanawalt (born 1931) is an American ...,Philip C. Hanawalt (born 1931) is an American ...,2,166,191,200 word wikipedia style introduction on 'Phil...,American graphic artist and illustrator. He i...,30,272
149998,2625970,https://en.wikipedia.org/wiki/Vossius%20Gymnasium,Vossius Gymnasium,Vossius Gymnasium is a public gymnasium in Ams...,Vossius Gymnasium is a public gymnasium in the...,2,168,108,200 word wikipedia style introduction on 'Voss...,"the town of Vossius, Netherlands. It is named...",32,147


# **Data Preprocessing**

In [52]:
df.drop(['url','title_len','wiki_intro_len','generated_intro_len','prompt','generated_text','prompt_tokens','generated_text_tokens'],axis=1,inplace=True)
print(df)

              id                     title  \
0       63064638    Sexhow railway station   
1         279621                  Etiäinen   
2         287229  Inverse function theorem   
3       26712375         Stepping on Roses   
4       38894426               Rob Bradley   
...          ...                       ...   
149995  44173767               Randy Borum   
149996  33564134           Sa'och language   
149997   4219548           Philip Hanawalt   
149998   2625970         Vossius Gymnasium   
149999  32345102           Simone Stratigo   

                                               wiki_intro  \
0       Sexhow railway station was a railway station b...   
1       In Finnish folklore, all places and things, an...   
2       In mathematics, specifically differential calc...   
3       is a Japanese shōjo manga series written and i...   
4       Robert Milner "Rob" Bradley, Jr. (born August ...   
...                                                   ...   
149995  Randy Borum 

## **Splitting the single dataframe into human generated and ai generated text,  adding the 'result' column**

In [53]:
dfh=df.copy()
dfh.drop(['generated_intro'],axis=1,inplace=True)
dfh=dfh.tail(50000)
dfh.rename(columns = {'wiki_intro':'text'}, inplace = True)
result=[]
for i in range(50000):
  result.append("Human Written")
dfh['result']=result
print(dfh)

              id              title  \
100000     66488       Moscow Metro   
100001   8281334    Delicata squash   
100002    509800      Sanjeev Kumar   
100003  22757288       Denny Somach   
100004  37911262     Luca Ciarrocca   
...          ...                ...   
149995  44173767        Randy Borum   
149996  33564134    Sa'och language   
149997   4219548    Philip Hanawalt   
149998   2625970  Vossius Gymnasium   
149999  32345102    Simone Stratigo   

                                                     text         result  
100000  The Moscow Metro is a metro system serving Mos...  Human Written  
100001  Delicata squash is a variety of winter squash ...  Human Written  
100002  Sanjeev Kumar (born Harihar Jethalal Jariwala;...  Human Written  
100003  Denny Somach is an American businessman, autho...  Human Written  
100004  Luca Ciarrocca is an Italian Internet entrepre...  Human Written  
...                                                   ...            ...  
149995

In [54]:
dfg=df.copy()
dfg.drop(['wiki_intro'],axis=1,inplace=True)
dfg=dfg.tail(50000)
dfg.rename(columns = {'generated_intro':'text'}, inplace = True)
result=[]
for i in range(50000):
  result.append("AI Written")
dfg['result']=result
print(dfg)

              id              title  \
100000     66488       Moscow Metro   
100001   8281334    Delicata squash   
100002    509800      Sanjeev Kumar   
100003  22757288       Denny Somach   
100004  37911262     Luca Ciarrocca   
...          ...                ...   
149995  44173767        Randy Borum   
149996  33564134    Sa'och language   
149997   4219548    Philip Hanawalt   
149998   2625970  Vossius Gymnasium   
149999  32345102    Simone Stratigo   

                                                     text      result  
100000  The Moscow Metro is a metro system serving Mos...  AI Written  
100001  Delicata squash is a variety of winter squash ...  AI Written  
100002  Sanjeev Kumar (born Harihar Jethalal Jariwala;...  AI Written  
100003  Denny Somach is an American businessman, autho...  AI Written  
100004  Luca Ciarrocca is an Italian Internet entrepre...  AI Written  
...                                                   ...         ...  
149995  Randy Borum is a Pr

## **Combining the two dataframes into a single dataset**

In [55]:
ds=pd.concat([dfh,dfg])
ds.shape

(100000, 4)

In [56]:
print(ds)

              id              title  \
100000     66488       Moscow Metro   
100001   8281334    Delicata squash   
100002    509800      Sanjeev Kumar   
100003  22757288       Denny Somach   
100004  37911262     Luca Ciarrocca   
...          ...                ...   
149995  44173767        Randy Borum   
149996  33564134    Sa'och language   
149997   4219548    Philip Hanawalt   
149998   2625970  Vossius Gymnasium   
149999  32345102    Simone Stratigo   

                                                     text         result  
100000  The Moscow Metro is a metro system serving Mos...  Human Written  
100001  Delicata squash is a variety of winter squash ...  Human Written  
100002  Sanjeev Kumar (born Harihar Jethalal Jariwala;...  Human Written  
100003  Denny Somach is an American businessman, autho...  Human Written  
100004  Luca Ciarrocca is an Italian Internet entrepre...  Human Written  
...                                                   ...            ...  
149995

# **Data Preparation**

## **Data Seperation as x and y**


In [57]:
y=ds['result']
print(y)

100000    Human Written
100001    Human Written
100002    Human Written
100003    Human Written
100004    Human Written
              ...      
149995       AI Written
149996       AI Written
149997       AI Written
149998       AI Written
149999       AI Written
Name: result, Length: 100000, dtype: object


In [58]:
x=ds.drop(['result'], axis=1)
print(x)

              id              title  \
100000     66488       Moscow Metro   
100001   8281334    Delicata squash   
100002    509800      Sanjeev Kumar   
100003  22757288       Denny Somach   
100004  37911262     Luca Ciarrocca   
...          ...                ...   
149995  44173767        Randy Borum   
149996  33564134    Sa'och language   
149997   4219548    Philip Hanawalt   
149998   2625970  Vossius Gymnasium   
149999  32345102    Simone Stratigo   

                                                     text  
100000  The Moscow Metro is a metro system serving Mos...  
100001  Delicata squash is a variety of winter squash ...  
100002  Sanjeev Kumar (born Harihar Jethalal Jariwala;...  
100003  Denny Somach is an American businessman, autho...  
100004  Luca Ciarrocca is an Italian Internet entrepre...  
...                                                   ...  
149995  Randy Borum is a Professor and Coordinator of ...  
149996  Sa'och (, also, "Sauch") is an endangered, 

## **Data Splitting**

In [59]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(ds['text'], ds['result'],test_size=0.3, random_state= 42)

In [60]:
x_train

126513    Charles Alexander Bruce (11 January 1793 – 23 ...
110406    Frank Shelby Groner (1877–1943) was a lawyer, ...
127322    Grigori Petrovitch Maximoff (1893–1950) was a ...
103699    The Women's Emancipation Union was founded by ...
115412    "Monotonous" is a popular song written by Tom ...
                                ...                        
106265    Zhang Kunsheng (; born August 1958) is a forme...
104886    The Academy of Ikalto  was an educational inst...
126820    The GWR 645 Class was a class of 2-6-0 steam l...
100860    Robert "Bob" Gordon Vogel (born 7 October 1981...
115795    A singer is a person who sings. Singer or Sing...
Name: text, Length: 70000, dtype: object

In [61]:
x_test

125721    Metanoetics (from  "conversion, repentance" fr...
130184    Violence is the sixth studio album from Editor...
119864    Cielo Jazmín Veizaga Arteaga (born 17 March 20...
126699    Stromberg v. California, 283 U.S. 359 (1931), ...
142991    Peripheral chemoreceptors (of the carotid and ...
                                ...                        
142648    Michael J. Harder (born February 8, 1973) is a...
136306    Lenny Lyles (January 26, 1936 – November 1, 19...
145466    A barrel murder was a method of "execution" us...
113724    Jose "Junjun" Antiquisa Capistrano, Jr., was t...
134122    Bishop's University  is a small Liberal arts c...
Name: text, Length: 30000, dtype: object

In [62]:
y_train

126513       AI Written
110406       AI Written
127322    Human Written
103699       AI Written
115412       AI Written
              ...      
106265    Human Written
104886       AI Written
126820       AI Written
100860    Human Written
115795    Human Written
Name: result, Length: 70000, dtype: object

In [63]:
y_test

125721       AI Written
130184       AI Written
119864    Human Written
126699       AI Written
142991       AI Written
              ...      
142648    Human Written
136306       AI Written
145466    Human Written
113724       AI Written
134122    Human Written
Name: result, Length: 30000, dtype: object

# **Model Building**

## **Linear Support vector Machine (LSVM)**

### **Importing necessary libraries**

In [64]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

### **Vectorize the text data using TF-IDF vectorizer**

In [65]:
tfidf = TfidfVectorizer()

In [66]:

X_train_tfidf = tfidf.fit_transform(x_train)
X_test_tfidf = tfidf.transform(x_test)

### **Model creation**

In [67]:
lsvm_model = LinearSVC(random_state=42)

### **Model training**

In [68]:
lsvm_model.fit(X_train_tfidf, y_train)

LinearSVC(random_state=42)

# **Making predictions on the testing set**

In [69]:
y_pred = lsvm_model.predict(X_test_tfidf)

# **Computing the accuracy score of the model**

In [70]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

Accuracy: 0.9290666666666667


# **Testing the model with a random text**

In [74]:
random_text = "India is a vast and diverse country located in South Asia, bordered by the Indian Ocean on the south, the Bay of Bengal on the east, and the Arabian Sea on the west. It is the world's second-most populous country and home to a rich and varied cultural heritage, including ancient religions, traditions, and languages. India has a rapidly growing economy and is considered a major player in global affairs. Its cuisine, arts, music, and cinema are popular worldwide, and the country is famous for its colorful festivals, such as Diwali and Holi. However, India also faces many challenges, such as poverty, inequality, and environmental issues, as it continues to work towards achieving its potential as a modern and thriving nation."

### **Vectorising the text with the same TfidfVectorizer used to fit the training data**

In [75]:
random_text_tfidf = tfidf.transform([random_text])

### **Using the trained LinearSVC model to make predictions**

In [76]:
prediction = lsvm_model.predict(random_text_tfidf)
print("Prediction:", prediction)

Prediction: ['AI Written']
